# Instalari

In [1]:
!pip install pandas

In [2]:
!pip install numpy

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.0 MB/s eta 0:00:00


In [4]:
!pip install spacy

In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
!pip install sentence-transformers

In [7]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Importuri

In [8]:
import pandas as pd
import numpy as np
import faiss
import spacy
from datasets import load_dataset
import re
from spacy.lang.en.stop_words import STOP_WORDS
from sentence_transformers import SentenceTransformer

# DataSet

In [9]:
# Încarcă modelul spaCy
nlp = spacy.load("en_core_web_sm")

# Funcție pentru împărțirea în propoziții și eliminarea stop words
def process_text_with_spacy(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    return sentences

In [10]:
# Încarcă dataset-ul
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0", download_mode="force_redownload")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [11]:
train_dataset = dataset['train']  # Reține primele 50 de articole

In [12]:
print(len(train_dataset))

print(type(train_dataset))

287113
<class 'datasets.arrow_dataset.Dataset'>


In [13]:
data = []
i = 1
n = 15 #Vreau doar 50 de articole
print(n)
while i <= n:
    print(i,"   ",train_dataset['highlights'][i])  # Verifică tipul elementului
    title = train_dataset['highlights'][i]  # Extrage titlul
    sentences = process_text_with_spacy(train_dataset['article'][i])  # Pre-procesare
    for sentence in sentences:
        data.append({"title": title, "sentence": sentence})
    i= i + 1

15
1     Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
2     NEW: "I thought I was going to die," driver says .
Man says pickup truck was folded in half; he just has cut on face .
Driver: "I probably had a 30-, 35-foot free fall"
Minnesota bridge collapsed during rush hour Wednesday .
3     Five small polyps found during procedure; "none worrisome," spokesman says .
President reclaims powers transferred to vice president .
Bush undergoes routine colonoscopy at Camp David .
4     NEW: NFL chief, Atlanta Falcons owner critical of Michael Vick's conduct .
NFL suspends Falcons quarterback indefinitely without pay .
Vick admits funding dogfighting operation but says he did not gamble .
Vick due in federal court Monday; future in NFL remains uncertain .
5     

In [14]:
print(len(data))

486


In [15]:
print("Titlt:  " ,train_dataset['highlights'][1])
print("Articol:")
train_dataset['article'][1]

Titlt:   Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Articol:


'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

# Model

In [16]:
# Încarcă modelul de embedding
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/169k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

# Embedinguri

In [17]:
# Listele pentru embedding-uri și metadate
sentences = [item['sentence'] for item in data]
titles = [item['title'] for item in data]

In [18]:
# Generare embedding-uri pentru propoziții
embeddings = model.encode(sentences, show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

# IndexFlatIP

In [19]:
# Dimensiunea embedding-urilor
dimension = embeddings.shape[1]

# Crearea indexului FlatIP
index = faiss.IndexFlatIP(dimension)

In [20]:
index.ntotal

0

In [21]:
from sklearn.preprocessing import normalize
embeddings = normalize(embeddings, norm='l2')

In [22]:
# Adăugarea embedding-urilor în index
index.add(embeddings)

In [23]:
index.ntotal

486

In [24]:
# Funcția de căutare
def search_query(query, top_k=5):
    query_embedding = model.encode([query])
    query_embedding = normalize(query_embedding, norm='l2')  # Normalizează embedding-ul query-ului

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(top_k):
        result = {
            "title": titles[indices[0][i]],
            "sentence": sentences[indices[0][i]],
            "score": distances[0][i]
        }
        results.append(result)
    return results

In [25]:
# Exemplu de interogare
query = "The lack of proper mental health care for inmates in Miami's pretrial detention facility contributes to their worsening conditions."
results = search_query(query)

# Output rezultate
for result in results:
    print(f"Titlu: {result['title']}")
    print(f"Propoziție: {result['sentence']}")
    print(f"Scor Similaritate: {result['score']}")
    print("-" * 50)

Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
Scor Similaritate: 0.7668324112892151
--------------------------------------------------
Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: Leifman says about one-third of all people in Miami-Dade county jails are mentally ill.
Scor Similaritate: 0.7397561073303223
--------------------------------------------------
Titlu: Mentally ill inm

# Index HNSW

In [38]:
# Dimensiunea embedding-urilor
dimension = embeddings.shape[1]
M = 30
# Crearea indexului FlatIP
index = faiss.IndexHNSWFlat(dimension, M)  # 32 este numărul de vecini apropiați (eficiență vs memorie)

In [39]:
index.ntotal

0

In [40]:
#Normalize embeding
faiss.normalize_L2(embeddings)

In [41]:
# Adăugarea embedding-urilor în index
index.add(embeddings)

In [42]:
index.ntotal

486

In [43]:
# Funcția de căutare
def search_query(query, top_k=5):
    query_embedding = model.encode([query])
    query_embedding = normalize(query_embedding, norm='l2')  # Normalizează embedding-ul query-ului

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(top_k):
        result = {
            "title": titles[indices[0][i]],
            "sentence": sentences[indices[0][i]],
            "score": distances[0][i]
        }
        results.append(result)
    return results

In [44]:
# Exemplu de interogare
query = "The lack of proper mental health care for inmates in Miami's pretrial detention facility contributes to their worsening conditions."
results = search_query(query)

# Output rezultate
for result in results:
    print(f"Titlu: {result['title']}")
    print(f"Propoziție: {result['sentence']}")
    print(f"Scor Similaritate: {result['score']}")
    print("-" * 50)

Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
Scor Similaritate: 0.4663352072238922
--------------------------------------------------
Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: Leifman says about one-third of all people in Miami-Dade county jails are mentally ill.
Scor Similaritate: 0.520487904548645
--------------------------------------------------
Titlu: Mentally ill inma

# Index IVF Initial

In [103]:
# Dimensiunea embedding-urilor
dimension = embeddings.shape[1]
M = 2
quantizer = faiss.IndexFlatL2(dimension)  # Folosit pentru a partitiona spațiul
index = faiss.IndexIVFFlat(quantizer, dimension, M, faiss.METRIC_L2)

In [104]:
index.nprobe = 1

In [105]:
index.ntotal

0

In [106]:
index.is_trained

False

In [107]:
index.train(embeddings)

In [108]:
index.is_trained

True

In [109]:
#Normalize embeding
faiss.normalize_L2(embeddings)

In [110]:
# Adăugarea embedding-urilor în index
index.add(embeddings)

In [111]:
index.ntotal

486

In [112]:
# Funcția de căutare
def search_query(query, top_k=5):
    query_embedding = model.encode([query])
    query_embedding = normalize(query_embedding, norm='l2')  # Normalizează embedding-ul query-ului

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(top_k):
        result = {
            "title": titles[indices[0][i]],
            "sentence": sentences[indices[0][i]],
            "score": distances[0][i]
        }
        results.append(result)
    return results

In [113]:
# Exemplu de interogare
query = "The lack of proper mental health care for inmates in Miami's pretrial detention facility contributes to their worsening conditions."
results = search_query(query)

# Output rezultate
for result in results:
    print(f"Titlu: {result['title']}")
    print(f"Propoziție: {result['sentence']}")
    print(f"Scor Similaritate: {result['score']}")
    print("-" * 50)

Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: Leifman says about one-third of all people in Miami-Dade county jails are mentally ill.
Scor Similaritate: 0.520487904548645
--------------------------------------------------
Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
Scor Similaritate: 0.6186426281929016
--------------------------------------------------
Titlu: Mentall

# INdex IVF imbunatatit

In [114]:
# Dimensiunea embedding-urilor
dimension = embeddings.shape[1]
M = 20
quantizer = faiss.IndexFlatL2(dimension)  # Folosit pentru a partitiona spațiul
index = faiss.IndexIVFFlat(quantizer, dimension, M, faiss.METRIC_L2)

index.nprobe = 5

In [115]:
index.ntotal

0

In [116]:
index.is_trained

False

In [117]:
index.train(embeddings)

In [118]:
index.is_trained

True

In [119]:
#Normalize embeding
faiss.normalize_L2(embeddings)
# Adăugarea embedding-urilor în index
index.add(embeddings)

In [120]:
index.ntotal

486

In [121]:
# Funcția de căutare
def search_query(query, top_k=5):
    query_embedding = model.encode([query])
    query_embedding = normalize(query_embedding, norm='l2')  # Normalizează embedding-ul query-ului

    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(top_k):
        result = {
            "title": titles[indices[0][i]],
            "sentence": sentences[indices[0][i]],
            "score": distances[0][i]
        }
        results.append(result)
    return results

In [122]:
# Exemplu de interogare
query = "The lack of proper mental health care for inmates in Miami's pretrial detention facility contributes to their worsening conditions."
results = search_query(query)

# Output rezultate
for result in results:
    print(f"Titlu: {result['title']}")
    print(f"Propoziție: {result['sentence']}")
    print(f"Scor Similaritate: {result['score']}")
    print("-" * 50)

Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
Scor Similaritate: 0.4663352072238922
--------------------------------------------------
Titlu: Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
Propoziție: Leifman says about one-third of all people in Miami-Dade county jails are mentally ill.
Scor Similaritate: 0.520487904548645
--------------------------------------------------
Titlu: Mentally ill inma